## Nombre: Sebastián Urbina

# CC3001 Primavera 2020 Tarea 4
# Impresión de fórmulas representadas mediante árboles
## Profesores
Sección 1 Benjamin Bustos • Sección 2 Jérémy Barbay • Sección 3 Patricio Poblete / Nelson Baloian

El objetivo de esta tarea es aprender a procesar la información de árboles que representan fórmulas. Hay muchas cosas que uno puede hacer con estos árboles, y en esta tarea nos enfocaremos en particular en imprimir la fórmula respectiva usando solo los paréntesis estrictamente necesarios (de acuerdo a las reglas indicadas en este enunciado).

Específicamente, el problema es: dado un árbol que representa una fórmula, imprimirla utilizando solo los paréntesis necesarios para que sea interpretada correctamente, de acuerdo a las reglas de prioridad y de asociatividad de operadores.

Los operadores de menor prioridad son la suma (``+``) y la resta (``-``), de igual prioridad entre sí, luego vienen la multiplicación (``*``) y la división (``/``), también de igual prioridad entre sí, y finalmente el operador de mayor prioridad es la elevación a potencia (``^``). Todos los operadores asocian hacia la izquierda, excepto la elevación a potencia, que asocia hacia la derecha. Para simplificar no usaremos el operador "menos unario".

La diferencia de prioridad hace que para imprimir este árbol no se necesiten paréntesis:

```
     +
    / \
   *   /     ==> a*b+c/d
  / \ / \
  a b c d
```

mientras que en este otro sí haya que parentizar:

```
     *
    / \
   +   -     ==> (a+b)*(c-d)
  / \ / \
  a b c d
```

La asociatividad hacia la izquierda entre operadores de igual prioridad hace que en este árbol la sub-fórmula del lado izquierdo no necesite paréntesis, pero la del lado derecho, sí:

```
     -
    / \
   +   -     ==> a+b-(c-d)
  / \ / \
  a b c d
```

En el caso de la elevación a potencia, la asociatividad es hacia la derecha, así que en ese caso es la sub-fórmula izquierda la que necesita paréntesis:


```
     ^
    / \
   ^   ^     ==> (a^b)^c^d
  / \ / \
  a b c d
```


El objetivo de esta tarea es que usted implemente una función que, dado un árbol ya construído, lo recorra para imprimir la fórmula respectiva usando solo los paréntesis estrictamente necesarios, de acuerdo a las reglas recién señaladas.

A continuación usted encontrará las clases ``Nodoi``, ``Nodoe`` y ``Arbol`` que implementan árboles que representan fórmulas. La clase ``Arbol`` viene con un constructor que recibe un string como parámetro, el cual contiene una fórmula, y este constructor se encarga de transformar la fórmula de string a árbol, de modo que usted no necesita preocuparse de eso. Para simplificar, la fórmula solo podrá contener variables de una sola letra, números de un solo dígito, y sin espacios.

Usted tiene que implementar para la clase ``Arbol`` y para los nodos una función ``infijo`` que al ser invocada retorne un string que contenga la fórmula respectiva en al notación usual (infijo) solo con los paréntesis estrictamente necesarios.

Para ayudarlo, la clase viene con una función ``infijo`` que retorna la fórmula **con todos los paréntesis posibles**. Usted tiene que reemplazarla por lo que realmente se pide.

In [125]:
class Nodoi:
    def __init__(self, izq, info, der):
        self.izq=izq
        self.info=info
        self.der=der
    def postorden(self):
        self.izq.postorden()   
        self.der.postorden()
        print(self.info, end=" ")
    def infijo(self):
        if self.info in "+-" and self.izq.info in "*/" and self.der.info in "*/": #al sumar/restar productos/cocientes, evitar paréntesis
            return self.izq.infijo()+self.info+self.der.infijo()

        elif self.info in "*/" and self.izq.info in "+-" and self.der.info in "+-": #al multiplicar/dividir sumas/restas respetar paréntesis
            return "("+self.izq.infijo()+")"+self.info+"("+self.der.infijo()+")"
        
        elif self.info in "*/" and self.der.info in "+-": #idem elif anterior 
            return self.izq.infijo()+self.info+"("+self.der.infijo()+")"

        elif self.info in "*/" and self.izq.info in "+-": #regla al multiplicar un numero por una suma/resta(distributividad(?))
            return "("+self.izq.infijo()+")"+self.info+self.der.infijo()

        elif self.info in "-" and self.der.info in "-": #regla al restar operandos negativos
            return self.izq.infijo()+self.info+"("+self.der.infijo()+")"

        elif self.info in "^" and self.izq.info in "^" and self.der.info in "^": #asociatividad potencia
            return "("+self.izq.infijo()+")"+self.info+self.der.infijo()
            
        elif self.info in "^" and (self.izq.info in "+-*/" or self.der.info in "+-*/"): #regla de la potencia
            if self.der.infijo().isdigit() or self.der.infijo().isalpha():
                return "("+self.izq.infijo()+")"+self.info+self.der.infijo()
            elif self.izq.infijo().isdigit() or self.izq.infijo().isalpha():
                return self.izq.infijo()+self.info+"("+self.der.infijo()+")"
            elif (self.izq.infijo().isdigit() or self.izq.infijo().isalpha()) and (self.der.infijo().isalpha() or self.der.infijo().isdigit()):
                return self.izq.infijo()+self.info+self.der.infijo()
            return "("+self.izq.infijo()+")"+self.info+"("+self.der.infijo()+")"

        return self.izq.infijo()+self.info+self.der.infijo()
       
class Nodoe:
    def __init__(self, info):
        self.info=info
    def postorden(self):
        print(self.info, end=" ")
    def infijo(self):
        return self.info

class Arbol:
    def __init__(self,formula):
        global k
        global s
        s=formula+";" # agregamos una marca de fin de la entrada
        k=0 # indica próximo caracter por procesar
        # definimos funciones para analizar la fórmula
        def expresion(): # retorna puntero a la raíz de un árbol que representa a la fórmula s
            global k
            global s
            a=factor()
            while s[k]=="+" or s[k]=="-":
                op=s[k]
                k+=1
                b=factor()
                a=Nodoi(a,op,b)
            return a
        def factor():
            global k
            global s
            a=termino()
            while s[k]=="*" or s[k]=="/":
                op=s[k]
                k+=1
                b=termino()
                a=Nodoi(a,op,b)
            return a
        def termino():
            global k
            global s
            a=primario()
            if s[k]=="^":
                op=s[k]
                k+=1
                b=termino()
                a=Nodoi(a,op,b)
            return a
        def primario(): # posible constante, variable o formula parentizada
            global k
            global s
            if s[k].isalpha() or s[k].isdigit():
                a=Nodoe(s[k])
                k+=1
                return a
            if s[k]=="(": # fórmula parentizada
                k+=1
                a=expresion()
                if s[k]!=")":
                    print("Error: Falta cierra paréntesis: "+formula[k:])
                    assert False
                k+=1
                return a
            print("Error: Falta variable, número o abre paréntesis: "+formula[k:])
            assert False
              
        a=expresion()
        if s[k]!=";":
            print("Error: Basura al final de la fórmula: "+formula[k:])
            assert False                
        self.raiz=a
            
    def postorden(self):
        print("Postorden:", end=" ")
        self.raiz.postorden()
        print()
    
    def infijo(self):
        return self.raiz.infijo()

A continuación probamos esta funcionalidad con varias fórmulas de ejemplo. Para ver si el árbol fue construido correctamente, también lo recorremos en postorden para imprimirlo en notación polaca. (**Importante**: La notación polaca no tiene nada que ver con lo que ustedes tienen que hacer, es solo una manera de poder imprimir el árbol generado para chequear que esté bien construido).
A continuación se imprime la fórmula en la notación de infijo generada.

In [110]:
def probar(formula):
    f=Arbol(formula)
    f.postorden()
    print("Infijo:", f.infijo())

In [111]:
probar("(a+b)*(c-d)")

Postorden: a b + c d - * 
Infijo: (a+b)*(c-d)


In [112]:
probar("a*(b+c)")

Postorden: a b c + * 
Infijo: a*(b+c)


In [113]:
probar("(a+b)-(c-d)")

Postorden: a b + c d - - 
Infijo: a+b-(c-d)


In [114]:
probar("(a^b)^(c^d)")


Postorden: a b ^ c d ^ ^ 
Infijo: (a^b)^c^d


## Pruebas adicionales

En esta sección se le pide que usted pruebe su programa con las fórmulas que se le entregan, y que invente dos ejemplos adicionales.

In [115]:
probar("a^b*c^d-e^f/g^h")

Postorden: a b ^ c d ^ * e f ^ g h ^ / - 
Infijo: a^b*c^d-e^f/g^h


In [116]:
probar("((a-b)*(c+d))^((e+f)/(g-h))")

Postorden: a b - c d + * e f + g h - / ^ 
Infijo: ((a-b)*(c+d))^((e+f)/(g-h))


In [117]:
probar("1+(2/3)^4")

Postorden: 1 2 3 / 4 ^ + 
Infijo: 1+(2/3)^4


In [118]:
probar("2^(4-a)^3*5")

Postorden: 2 4 a - 3 ^ ^ 5 * 
Infijo: 2^(4-a)^3*5


In [119]:
probar("1+4^(2*4)")

Postorden: 1 4 2 4 * ^ + 
Infijo: 1+4^(2*4)


In [120]:
probar("(a*b)+c^(2/4)+1")

Postorden: a b * c 2 4 / ^ + 1 + 
Infijo: a*b+c^(2/4)+1
